# Causal Discovery for datasets with mixed feature types


In [1]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import pgmpy
from pgmpy.estimators import PC, BicScore, HillClimbSearch
from pgmpy.models import BayesianNetwork
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Data preparation


In [2]:
from openml import datasets

# dataset = datasets.get_dataset(179)
dataset = datasets.get_dataset(14)

In [3]:
df = dataset.get_data(target=dataset.default_target_attribute)[0]
df.info()

INFO:openml.datasets.dataset:pickle load data mfeat-fourier


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 76 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   att1    2000 non-null   float64
 1   att2    2000 non-null   float64
 2   att3    2000 non-null   float64
 3   att4    2000 non-null   float64
 4   att5    2000 non-null   float64
 5   att6    2000 non-null   float64
 6   att7    2000 non-null   float64
 7   att8    2000 non-null   float64
 8   att9    2000 non-null   float64
 9   att10   2000 non-null   float64
 10  att11   2000 non-null   float64
 11  att12   2000 non-null   float64
 12  att13   2000 non-null   float64
 13  att14   2000 non-null   float64
 14  att15   2000 non-null   float64
 15  att16   2000 non-null   float64
 16  att17   2000 non-null   float64
 17  att18   2000 non-null   float64
 18  att19   2000 non-null   float64
 19  att20   2000 non-null   float64
 20  att21   2000 non-null   float64
 21  att22   2000 non-null   float64
 22  

In [4]:
# === Preprocess the data ===
# Encode the categorical data
cat_encoder = OneHotEncoder()
cat_data = cat_encoder.fit_transform(df.select_dtypes(include=["object", "category"]))
cat_data = pd.DataFrame(
    cat_data.toarray(),
    columns=cat_encoder.get_feature_names_out(df.select_dtypes(include=["object", "category"]).columns),
)
df = df.drop(columns=df.select_dtypes(include=["object", "category"]).columns)
df = pd.concat([df, cat_data], axis=1)

df

att1      att2      att3      att4      att5      att6      att7  \
0     0.065882  0.197312  0.103826  0.270362  0.616078  0.035856  0.424572   
1     0.049142  0.175971  0.105515  0.227095  0.599280  0.041217  0.431078   
2     0.034172  0.227649  0.108766  0.127697  0.612494  0.056554  0.470639   
3     0.062336  0.217979  0.080243  0.289592  0.546316  0.045779  0.425545   
4     0.061970  0.198358  0.111239  0.253460  0.608455  0.023631  0.415246   
...        ...       ...       ...       ...       ...       ...       ...   
1995  0.171874  0.221525  0.426470  0.077220  0.264413  0.220274  0.452574   
1996  0.152025  0.347439  0.323534  0.087706  0.427077  0.313656  0.371549   
1997  0.335606  0.318426  0.257948  0.225381  0.417769  0.123067  0.333455   
1998  0.253910  0.171202  0.302622  0.181066  0.326383  0.167960  0.417409   
1999  0.271575  0.149037  0.232753  0.197720  0.236424  0.187395  0.467213   

          att8      att9     att10  ...     att67     att68     att69  \
0     0.089701  0.367773  0.037065  ...  0.123448  0.113374  0.074343   
1     0.096801  0.326739  0.059661  ...  0.045310  0.069337  0.045386   
2     0.041903  0.324267  0.044569  ...  0.019858  0.218842  0.041087   
3     0.022841  0.331454  0.119052  ...  0.083995  0.085479  0.087658   
4     0.091866  0.309310  0.049142  ...  0.021004  0.106960  0.032283   
...        ...       ...       ...  ...       ...       ...       ...   
1995  0.161113  0.356788  0.081889  ...  0.108996  0.197440  0.197778   
1996  0.262768  0.396422  0.124391  ...  0.212160  0.161606  0.033621   
1997  0.342044  0.398151  0.079550  ...  0.236628  0.121659  0.316333   
1998  0.399065  0.266153  0.097131  ...  0.155442  0.168154  0.147167   
1999  0.385117  0.191292  0.149162  ...  0.140143  0.225634  0.061724   

         att70     att71     att72     att73     att74     att75     att76  
0     0.281666  0.067795  0.344182  0.038963  0.394366  0.049971  0.344871  
1     0.254264  0.045447  0.335659  0.021719  0.445277  0.083978  0.354092  
2     0.360464  0.047154  0.377408  0.052099  0.445029  0.071234  0.261465  
3     0.151880  0.075950  0.293462  0.022675  0.408291  0.063010  0.401376  
4     0.248565  0.015674  0.386276  0.039481  0.434701  0.069218  0.405403  
...        ...       ...       ...       ...       ...       ...       ...  
1995  0.205866  0.138887  0.160848  0.126190  0.247345  0.239117  0.136807  
1996  0.173155  0.295227  0.259417  0.345691  0.068303  0.158352  0.282327  
1997  0.244846  0.132893  0.042988  0.250740  0.141576  0.038776  0.114591  
1998  0.268675  0.199125  0.128708  0.285220  0.230322  0.103664  0.130003  
1999  0.267944  0.251182  0.170772  0.322067  0.275769  0.021074  0.085241  

[2000 rows x 76 columns]

## PC algorithm


In [5]:
# estimated_pdag = PC(df).estimate(ci_test='pillai')
# estimated_pdag.to_graphviz().draw('adult_pillai.png', prog='dot')

## MMHC algorithm

In [6]:
from pgmpy.estimators import MmhcEstimator

est = MmhcEstimator(df)
model = est.estimate()

KeyboardInterrupt: 

## Tree search algorithm

In [9]:
from pgmpy.estimators import TreeSearch

est = TreeSearch(df)
model = est.estimate()
model.to_graphviz().draw('adult_tree.png', prog='dot')

Building tree:   0%|          | 0/2850.0 [00:00<?, ?it/s]

/home/xj265/.local/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/home/xj265/.local/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/home/xj265/.local/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/home/xj265/.local/lib/python3.10/site-packages/sklearn/metrics/cluster/_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings

## Hill-Climb Search

In [ ]:
from pgmpy.estimators import HillClimbSearch

est = HillClimbSearch(df)
model = est.estimate()
model.to_graphviz().draw('adult_hill.png', prog='dot')

  0%|          | 0/1000000 [00:00<?, ?it/s]

/home/xj265/.local/lib/python3.10/site-packages/pgmpy/estimators/StructureScore.py:136: RuntimeWarning: overflow encountered in scalar multiply
  (num_parents_states - counts.shape[1]) * var_cardinality * gammaln(1)
/home/xj265/.local/lib/python3.10/site-packages/pgmpy/estimators/StructureScore.py:136: RuntimeWarning: overflow encountered in scalar multiply
  (num_parents_states - counts.shape[1]) * var_cardinality * gammaln(1)
/home/xj265/.local/lib/python3.10/site-packages/pgmpy/estimators/StructureScore.py:136: RuntimeWarning: overflow encountered in scalar multiply
  (num_parents_states - counts.shape[1]) * var_cardinality * gammaln(1)
/home/xj265/.local/lib/python3.10/site-packages/pgmpy/estimators/StructureScore.py:136: RuntimeWarning: overflow encountered in scalar multiply
  (num_parents_states - counts.shape[1]) * var_cardinality * gammaln(1)
/home/xj265/.local/lib/python3.10/site-packages/pgmpy/estimators/StructureScore.py:136: RuntimeWarning: overflow encountered in scalar m

## LiM algorithm


In [ ]:
import lingam

model = lingam.LiM()
dis_con = np.full((1, df.shape[1]), 0)
dis_con[0, [0, 1]] = 1
model.fit(df.values, dis_con=dis_con, only_global=True)

/home/xj265/.local/lib/python3.10/site-packages/lingam/lim.py:119: RuntimeWarning: overflow encountered in cosh
  loss_con = np.sum(-np.log(np.cosh(R)) * dis_con)
/home/xj265/.local/lib/python3.10/site-packages/lingam/lim.py:119: RuntimeWarning: invalid value encountered in multiply
  loss_con = np.sum(-np.log(np.cosh(R)) * dis_con)
/home/xj265/.local/lib/python3.10/site-packages/numpy/linalg/linalg.py:680: RuntimeWarning: overflow encountered in matmul
  result = z if result is None else fmatmul(result, z)
/home/xj265/.local/lib/python3.10/site-packages/numpy/linalg/linalg.py:677: RuntimeWarning: overflow encountered in matmul
  z = a if z is None else fmatmul(z, z)
/home/xj265/.local/lib/python3.10/site-packages/lingam/lim.py:251: RuntimeWarning: invalid value encountered in multiply
  h = (G_h.T * M).sum() - d


KeyboardInterrupt: 